In [1]:
import pandas as pd
from psycopg2 import connect, DatabaseError, OperationalError

In [2]:
HOST = 'localhost'
USER = 'postgres'
PASSWORD = ''
DATABASE = 'bi_db_aje_co'

In [3]:
conn = None

In [4]:
conn = None
# Connect to get sales data
try:  # Create connection and build dataset
    conn = connect(
        host=HOST,
        database=DATABASE,
        user=USER,
        password=PASSWORD
    )
    print("Connection is successful.")
    df_sales = pd.read_sql("select fecha_de_visita, codigo_de_cliente, codigo_de_producto, venta_neta_dolar "
                           "from public.sales where venta_neta_dolar > 0.00", conn, coerce_float=False)
except (Exception, DatabaseError, OperationalError) as e:
    print("Failed to connect and process data, error is", str(e.args))
    if conn:
        conn.close()
    exit(45)
finally:
    if conn:
        conn.close()

Failed to connect and process data, error is ('fe_sendauth: no password supplied\n',)


In [10]:
df_sales = {
    'fecha_de_visita': ['2021-05-07', '2021-05-07', '2021-05-07', '2021-05-07', '2021-05-07', '2021-05-07'],
    'codigo_de_cliente': ['0094100823', '0094100823', '0094100823', '0094100823', '0094101546', '0094101546'],
    'codigo_de_producto': ['500040', '500366', '500574', '500575', '501323', '500040'],
    'venta_neta_dolar': [1.2, 2.2, 0, 2.11, 15.0, 0]
}

In [5]:
sample_input_data = {
    'fecha_de_visita': ['2021-05-07', '2021-05-07', '2021-05-07', '2021-05-07', '2021-05-07', '2021-05-07'],
    'codigo_de_cliente': ['0094100823', '0094100823', '0094100823', '0094100823', '0094101546', '0094101546'],
    'codigo_de_producto': ['500040', '500366', '500574', '500575', '501323', '500040'],
    'prod_rank': [1, 2, 3, 4, 1, 2]
}

In [11]:
sample_df = pd.DataFrame(sample_input_data)
df_sales = pd.DataFrame(df_sales)

In [12]:
sample_df

,fecha_de_visita,codigo_de_cliente,codigo_de_producto,prod_rank
0,2021-05-07,0094100823,500040,1
1,2021-05-07,0094100823,500366,2
2,2021-05-07,0094100823,500574,3
3,2021-05-07,0094100823,500575,4
4,2021-05-07,0094101546,501323,1
5,2021-05-07,0094101546,500040,2


In [14]:
df_sales

,fecha_de_visita,codigo_de_cliente,codigo_de_producto,venta_neta_dolar
0,2021-05-07,0094100823,500040,1.20
1,2021-05-07,0094100823,500366,2.20
2,2021-05-07,0094100823,500574,0.00
3,2021-05-07,0094100823,500575,2.11
4,2021-05-07,0094101546,501323,15.00
5,2021-05-07,0094101546,500040,0.00


In [13]:
print(sample_df.dtypes)
print(df_sales.dtypes)

fecha_de_visita       object
codigo_de_cliente     object
codigo_de_producto    object
prod_rank              int64
dtype: object
fecha_de_visita        object
codigo_de_cliente      object
codigo_de_producto     object
venta_neta_dolar      float64
dtype: object


In [15]:
sample_df.fecha_de_visita = sample_df.fecha_de_visita.astype(str)
sample_df.codigo_de_cliente = sample_df.codigo_de_cliente.astype(str)
sample_df.codigo_de_producto = sample_df.codigo_de_producto.astype(str)
df_sales.fecha_de_visita = df_sales.fecha_de_visita.astype(str)
df_sales.codigo_de_cliente = df_sales.codigo_de_cliente.astype(str)
df_sales.codigo_de_producto = df_sales.codigo_de_producto.astype(str)

In [16]:
joined_dataset = pd.merge(sample_df, df_sales, how='left',
                          left_on=['fecha_de_visita', 'codigo_de_cliente', 'codigo_de_producto'],
                          right_on=['fecha_de_visita', 'codigo_de_cliente', 'codigo_de_producto'],
                          suffixes=('_recomm', '_sales'))

In [17]:
joined_dataset

,fecha_de_visita,codigo_de_cliente,codigo_de_producto,prod_rank,venta_neta_dolar
0,2021-05-07,0094100823,500040,1,1.20
1,2021-05-07,0094100823,500366,2,2.20
2,2021-05-07,0094100823,500574,3,0.00
3,2021-05-07,0094100823,500575,4,2.11
4,2021-05-07,0094101546,501323,1,15.00
5,2021-05-07,0094101546,500040,2,0.00


In [18]:
joined_dataset['venta_neta_dolar'] = joined_dataset['venta_neta_dolar'].fillna(0.00)

In [19]:
joined_dataset

,fecha_de_visita,codigo_de_cliente,codigo_de_producto,prod_rank,venta_neta_dolar
0,2021-05-07,0094100823,500040,1,1.20
1,2021-05-07,0094100823,500366,2,2.20
2,2021-05-07,0094100823,500574,3,0.00
3,2021-05-07,0094100823,500575,4,2.11
4,2021-05-07,0094101546,501323,1,15.00
5,2021-05-07,0094101546,500040,2,0.00


In [20]:
joined_dataset['executed_flag'] = joined_dataset['venta_neta_dolar'].apply(lambda x: 1 if x > 0.00 else 0)

In [21]:
joined_dataset

,fecha_de_visita,codigo_de_cliente,codigo_de_producto,prod_rank,venta_neta_dolar,executed_flag
0,2021-05-07,0094100823,500040,1,1.20,1
1,2021-05-07,0094100823,500366,2,2.20,1
2,2021-05-07,0094100823,500574,3,0.00,0
3,2021-05-07,0094100823,500575,4,2.11,1
4,2021-05-07,0094101546,501323,1,15.00,1
5,2021-05-07,0094101546,500040,2,0.00,0


In [22]:
client_level_precision = joined_dataset.groupby(['fecha_de_visita', 'codigo_de_cliente']).agg({'codigo_de_producto': 'count', 'executed_flag': 'sum'}).reset_index()


In [23]:
client_level_precision

,fecha_de_visita,codigo_de_cliente,codigo_de_producto,executed_flag
0,2021-05-07,0094100823,4,3
1,2021-05-07,0094101546,2,1


In [24]:
client_level_precision['precision'] = client_level_precision['executed_flag']/client_level_precision['codigo_de_producto']


In [25]:
client_level_precision

,fecha_de_visita,codigo_de_cliente,codigo_de_producto,executed_flag,precision
0,2021-05-07,0094100823,4,3,0.75
1,2021-05-07,0094101546,2,1,0.50


In [26]:
precision = client_level_precision['precision'].mean()

In [27]:
precision

0.625